In [20]:
import time
from spade.agent import Agent
from spade.behaviour import CyclicBehaviour, PeriodicBehaviour, OneShotBehaviour, FSMBehaviour, State
from spade.message import Message
from spade.template import Template
import json
import asyncio
from random import random
from copy import deepcopy
import pandas as pd

In [6]:
from datetime import datetime

today = datetime.today()
print("Today's date:", today)

Today's date: 2022-06-12 14:12:24.970039


In [7]:
today

datetime.datetime(2022, 6, 12, 14, 12, 24, 970039)

In [8]:
password="Secret777"

In [9]:
jid ="SystemAgent@jabbim.pl"

In [10]:
class MachineAgent(Agent):

    def __init__(self, jid, password, verify_security, inputs, outputs, production_time, chance_to_break, magazine, item_prices, euro, up_price, down_price, providers, maintainers, stock):
        self.inputs=inputs
        self.outputs=outputs
        self.production_time=production_time
        self.chance_to_break=chance_to_break
        self.magazine=magazine
        self.item_prices=item_prices
        self.euro=euro
        self.up_price=up_price
        self.down_price=down_price
        self.providers=providers
        self.maintainers=maintainers
        self.stock=stock
        self.needed={item:self.inputs[item]-self.magazine[item] for item in self.inputs if self.magazine[item]<self.inputs[item]}
        self.start_production=True
        self.working_status=True
        
        super().__init__(jid, password, verify_security=verify_security)

    class ProduceBehav(CyclicBehaviour):
        async def run(self):
            await asyncio.sleep(1)
            self.agent.needed={item:self.agent.inputs[item]-self.agent.magazine[item] for item in self.agent.inputs if self.agent.magazine[item]<self.agent.inputs[item]}
            if(random()<self.agent.chance_to_break or self.agent.working_status==False):
                if self.agent.working_status:
                    print("broken")

                self.agent.working_status=False
                for maintainer in self.agent.maintainers:
                    msg = Message(to=maintainer)
                    msg.set_metadata("repair", "request")
                    await self.send(msg)
                    print("can you repair me?")
                    msg = await self.receive(timeout=10)
                    if msg:
                        if msg.get_metadata("repair")=="accepted":
                            print("waiting for repair")     
                            msg = await self.receive(timeout=10)
                            if msg:
                                if msg.get_metadata("repair")=="machine":
                                    self.agent.working_status=True
                                    print("Thank you for repair")
                                    break
                    else:
                        print("answer not received")

            if self.agent.start_production and self.agent.working_status and len(self.agent.needed)==0:
                await asyncio.sleep(self.agent.production_time)
                for item in self.agent.inputs:
                    self.agent.magazine[item]=self.agent.magazine[item]-self.agent.inputs[item]
                for item in self.agent.outputs:
                    self.agent.magazine[item]=self.agent.magazine[item]+self.agent.outputs[item]

                self.agent.stock.notify(str(self.agent.jid),datetime.today(),self.agent.magazine)

    class BuyBehav(CyclicBehaviour):

        async def run(self):
            propositions_list=[]
            if len(self.agent.needed)>0:
                for item in self.agent.needed:
                    for Provider in self.agent.providers[item]:
                        msg = Message(to=Provider)
                        msg.set_metadata("buy", "order")
                        msg.body = json.dumps({"item": item,"quantity":str(self.agent.needed[item])})
                        await self.send(msg)
                        print(Provider+" please "+item)
                        
                    
                    while len(propositions_list)<len(self.agent.providers[item]):
                        msg = await self.receive(timeout=10)
                        propositions_list.append(msg)
                    
                    best_price=100000
                    best_proposition=None
                    for msg in propositions_list:
                        if msg:
                            if msg.get_metadata("sell")=="proposition":
                                print("Message received with content: {}".format(msg.body))
                                body = json.loads(msg.body)
                                get_item = body["item"]
                                get_payment = int(body["payment"])
                                if get_payment<best_price:
                                    best_proposition=msg
                                    best_price=get_payment
                                
                    if best_proposition:
                        msg = Message(to=str(best_proposition.sender))
                        msg.set_metadata("buy", "payment")
                        if item in self.agent.needed.keys() and self.agent.euro>=best_price:
                            msg.body = json.dumps({"item": item, "quantity":str(self.agent.needed[item]),"payment":str(best_price)})
                            self.agent.euro-=best_price
                            await self.send(msg)
                            print("sending cash")
                            
                            msg = await self.receive(timeout=10)
                            if msg:
                                if msg.get_metadata("sell")=="product":
                                    body = json.loads(msg.body)
                                    get_item = body["item"]
                                    get_quantity = int(body["quantity"])

                                    self.agent.magazine[item]=self.agent.magazine[item]+get_quantity
                                    self.agent.stock.notify(str(self.agent.jid),datetime.today(),self.agent.magazine)
                                    print("thank you " + str(best_proposition.sender))   

            

    class SellBehav(CyclicBehaviour):
        async def run(self):
            #await asyncio.sleep(1)
            msg = await self.receive(timeout=10)
            if msg:
                get_type=msg.get_metadata("buy")
                body = json.loads(msg.body)
                get_item = body["item"]
                get_quantity = int(body["quantity"])
                if get_type=="order":
                    if self.agent.magazine[get_item]>=get_quantity:
                        msg = Message(to=str(msg.sender))
                        msg.set_metadata("sell", "proposition")
                        #update price
                        print("Changing price from "+str(self.agent.item_prices[get_item])+" to "+str(int(self.agent.item_prices[get_item]*self.agent.down_price)))
                        self.agent.item_prices[get_item]=int(self.agent.item_prices[get_item]*self.agent.down_price)
                        
                        msg.body = json.dumps({"item": get_item,"payment":str(self.agent.item_prices[get_item]*get_quantity)})
                        await self.send(msg)
                        print("proposing")
                    else:
                        msg = Message(to=str(msg.sender))
                        msg.set_metadata("sell", "decline")
                        msg.body = json.dumps({"item": get_item, "quantity":0})
                        await self.send(msg)
                        print("no item now")
                        self.agent.start_production=True
                        
                if get_type=="payment":
                    get_payment = int(body["payment"])                    
                    self.agent.euro+=get_payment
                    self.agent.magazine[get_item]=self.agent.magazine[get_item]-get_quantity
                    self.agent.stock.notify(str(self.agent.jid),datetime.today(),self.agent.magazine)
                    msg = Message(to=str(msg.sender))
                    msg.set_metadata("sell", "product")
                    msg.body = json.dumps({"item": get_item,"quantity":str(get_quantity)})
                    
                    #update price
                    print("up price")
                    self.agent.item_prices[get_item]=int(self.agent.item_prices[get_item]*self.agent.up_price)
                    
                    await self.send(msg)
                    print("sending product")
                    
            else:
                print("waiting for sell")

    async def setup(self):
        print(f"MachineAgent started at {datetime.now().time()}")
        t_repair = Template()
        t_repair.set_metadata("repair", "machine")
        t_repair_decline = Template()
        t_repair_decline.set_metadata("repair", "declined")
        t_repair_accepted = Template()
        t_repair_accepted.set_metadata("repair", "accepted")
        t_product = Template()
        t_product.set_metadata("sell", "product")
        t_decline = Template()
        t_decline.set_metadata("sell", "decline")
        t_proposition = Template()
        t_proposition.set_metadata("sell", "proposition")
        t_order = Template()
        t_order.set_metadata("buy", "order")
        t_payment = Template()
        t_payment.set_metadata("buy", "payment")
        self.p = self.ProduceBehav()
        self.add_behaviour(self.p,t_repair | t_repair_decline | t_repair_accepted)
        self.b = self.BuyBehav()
        self.add_behaviour(self.b,t_product | t_decline | t_proposition)
        self.s = self.SellBehav()
        self.add_behaviour(self.s,t_payment | t_order)
        await asyncio.sleep(5)

In [11]:
class OrderAgent(Agent):

    def __init__(self, jid: str, password: str, verify_security: bool, need, magazine, request_time, max_item_price, providers, stock):
        self.need=need
        self.providers=providers
        self.magazine=magazine
        self.request_time=request_time
        self.max_item_price=max_item_price
        self.stock=stock
        super().__init__(jid, password, verify_security=verify_security)

    class GetBehav(CyclicBehaviour):
        async def run(self):
            
            await asyncio.sleep(self.agent.request_time)
            item=self.agent.need
            propositions_list=[]
            for Provider in self.agent.providers[item]:
                msg = Message(to=Provider)
                msg.set_metadata("buy", "order")
                msg.body = json.dumps({"item": item,"quantity":1})
                await self.send(msg)
                print(Provider+" please "+item)


            while len(propositions_list)<len(self.agent.providers[item]):
                msg = await self.receive(timeout=10)
                propositions_list.append(msg)


            best_price=100000
            best_proposition=None
            for msg in propositions_list:
                if msg:
                    if msg.get_metadata("sell")=="proposition":
                        print("Message received with content: {}".format(msg.body))
                        body = json.loads(msg.body)
                        get_item = body["item"]
                        get_payment = int(body["payment"])
                        if get_payment<best_price and get_payment<=self.agent.max_item_price:
                            best_proposition=msg
                            best_price=get_payment

            if best_proposition:
                msg = Message(to=str(best_proposition.sender))
                msg.set_metadata("buy", "payment")
                msg.body = json.dumps({"item": item, "quantity":1,"payment":str(best_price)})
                await self.send(msg)
                print("sending cash "+str(best_price))
                msg = await self.receive(timeout=10)
                if msg:
                    body = json.loads(msg.body)
                    get_item = body["item"]
                    get_quantity = int(body["quantity"])
                    self.agent.magazine[item]=self.agent.magazine[item]+get_quantity
                    self.agent.stock.notify(str(self.agent.jid),datetime.today(),self.agent.magazine)
                    print("thank you " + str(best_proposition.sender))


    async def setup(self):
        print(f"OrderAgent started at {datetime.now().time()}")
        t_product = Template()
        t_product.set_metadata("sell", "product")
        t_decline = Template()
        t_decline.set_metadata("sell", "decline")
        t_proposition = Template()
        t_proposition.set_metadata("sell", "proposition")
        self.b = self.GetBehav()
        self.add_behaviour(self.b,t_product | t_decline | t_proposition)
        await asyncio.sleep(5)

In [12]:
class MaintenanceAgent(Agent):
    def __init__(self, jid: str, password: str, verify_security: bool, repair_time):
        self.repair_time=repair_time
        self.busy=False
        self.client=None
        super().__init__(jid, password, verify_security=verify_security)
    

    class RepairBehav(CyclicBehaviour):
        async def run(self):
            await asyncio.sleep(1)
            #print(self.agent.client)
            if self.agent.busy and self.agent.client!=None:
                await asyncio.sleep(self.agent.repair_time)
                msg = Message(to=self.agent.client)
                msg.set_metadata("repair", "machine")
                print("Repairing "+self.agent.client)
                await self.send(msg)
                self.agent.busy=False

    class ListenBehav(CyclicBehaviour):
        async def run(self):
            await asyncio.sleep(1)
            msg = await self.receive(timeout=10)
            if msg:
                if msg.get_metadata("repair") and self.agent.busy==False:
                    print("repair accepted")
                    self.agent.client=str(msg.sender)
                    msg = Message(to=str(msg.sender))
                    msg.set_metadata("repair", "accepted")
                    await self.send(msg)
                    self.agent.busy=True
                else:
                    msg = Message(to=str(msg.sender))
                    msg.set_metadata("repair", "declined")
                    await self.send(msg)

    async def setup(self):
        print(f"MaintenanceAgent started at {datetime.now().time()}")
        t_repair = Template()
        t_repair.set_metadata("repair", "request")
        self.l = self.ListenBehav()
        self.add_behaviour(self.l,t_repair)
        self.r = self.RepairBehav()
        self.add_behaviour(self.r)

In [13]:
class Stock():

    def __init__(self,agent_magazines):
        self.agent_magazines=agent_magazines

    def notify(self,agent,timestamp,magazine):
        self.agent_magazines[agent].append((timestamp,deepcopy(magazine)))
        
    def flatten(self):
        data=self.agent_magazines
        flat=[]
        for a in data:
            for r in data[a]:
                time=r[0]
                for p in r[1]:
                    flat.append({"time":time,"product":p,"quantity":r[1][p],"agent":a})

        df=pd.DataFrame(flat)
        return df

In [14]:
class Factory():
    def __init__(self,jid,password):
        self.jid=jid
        self.password=password
        self.max_id=0
        
    def load_agents(self,factory_setting):
        with open(factory_setting) as json_file:
            data = json.load(json_file)
        json_file.close()
        
        maintainer_list=[]
        for ma in data["Maintainers"]:
            for i in range(ma["number"]):
                self.max_id+=1
                maintainer_list.append(MaintenanceAgent(self.jid+"/"+str(self.max_id), self.password, False, ma["repair_time"]))
        
        machine_list=[]
        for m in data["Machines"]:
            for i in range(m["number"]):
                self.max_id+=1
                machine_list.append(MachineAgent(self.jid+"/"+str(self.max_id), self.password, False, m["inputs"],m["outputs"],m["production_time"],m["chance_to_break"],deepcopy(m["magazine"]),deepcopy(m["item_prices"]),m["euro"],m["up_price"],m["down_price"],None,maintainer_list,None))
        
        Providers={}
        
        for m in machine_list:
            for p in m.outputs:
                if p in Providers:
                    Providers[p].append(str(m.jid))
                else:
                    Providers[p]=[str(m.jid)]
                
        for m in machine_list:
            m.providers={p:Providers[p] for p in m.inputs if p in Providers}
        
        order_list=[]
        for o in data["Orders"]:
            for i in range(o["number"]):
                self.max_id+=1
                order_list.append(OrderAgent(self.jid+"/"+str(self.max_id), self.password, False, o["need"], deepcopy(o["magazine"]), o["request_time"],o["max_item_price"], None, None))

        for order in order_list:
            order.providers={p:Providers[p] for p in [order.need] if p in Providers}
            
        stock=Stock({str(m.jid):[] for m in machine_list+order_list})
        for m in machine_list+order_list:
            m.stock=stock
            
        self.Stock=stock
        self.Providers=Providers
        self.maintainer_list=maintainer_list
        self.machine_list=machine_list
        self.order_list=order_list
        
    def run_agents(self,timeout=30):
        
        try:
            for ma in self.maintainer_list:
                ma.start()
                
            for m in self.machine_list:
                m.start()
            
            for o in self.order_list:
                o.start()
            
            time.sleep(timeout)
        except Exception as e:
            print(e)
        finally:
            print("STOPPING!")
            for ag in self.maintainer_list+self.machine_list+self.order_list:
                ag.stop()          

In [15]:
factory=Factory(jid,password)

In [16]:
factory.load_agents('Factory1.json')

In [17]:
factory.Providers

{'milk': ['systemagent@jabbim.pl/2', 'systemagent@jabbim.pl/3']}

In [18]:
factory.run_agents()

MachineAgent started at 14:12:32.517592
OrderAgent started at 14:12:32.531035
MachineAgent started at 14:12:32.547874
OrderAgent started at 14:12:32.621282
MaintenanceAgent started at 14:12:32.642159
OrderAgent started at 14:12:32.703473
systemagent@jabbim.pl/2 please milk
systemagent@jabbim.pl/3 please milk
no item now
no item now
systemagent@jabbim.pl/2 please milk
systemagent@jabbim.pl/3 please milk
no item now
no item now
systemagent@jabbim.pl/2 please milk
systemagent@jabbim.pl/3 please milk
no item now
no item now
systemagent@jabbim.pl/2 please milk
systemagent@jabbim.pl/3 please milk
Changing price from 400 to 360
proposing
no item now
Message received with content: {"item": "milk", "payment": "360"}
sending cash 360
up price
sending product
thank you systemagent@jabbim.pl/2
systemagent@jabbim.pl/2 please milk
systemagent@jabbim.pl/3 please milk
no item now
Changing price from 400 to 360
proposing
Message received with content: {"item": "milk", "payment": "360"}
sending cash 360

systemagent@jabbim.pl/2 please milk
systemagent@jabbim.pl/3 please milk
Changing price from 531 to 477
proposing
Changing price from 590 to 531
proposing
Message received with content: {"item": "milk", "payment": "477"}
Message received with content: {"item": "milk", "payment": "531"}
sending cash 477
up price
sending product
thank you systemagent@jabbim.pl/2
systemagent@jabbim.pl/2 please milk
systemagent@jabbim.pl/3 please milk
no item now
Changing price from 531 to 477
proposing
Message received with content: {"item": "milk", "payment": "477"}
sending cash 477
up price
sending product
thank you systemagent@jabbim.pl/3
systemagent@jabbim.pl/2 please milk
systemagent@jabbim.pl/3 please milk
no item now
no item now
systemagent@jabbim.pl/2 please milk
systemagent@jabbim.pl/3 please milk
no item now
no item now
systemagent@jabbim.pl/2 please milk
systemagent@jabbim.pl/3 please milk
no item now
no item now
systemagent@jabbim.pl/2 please milk
systemagent@jabbim.pl/3 please milk
no item now

No behaviour matched for message: <message to="systemagent@jabbim.pl/3" from="systemagent@jabbim.pl/5" thread="None" metadata={'buy': 'payment'}>
{"item": "milk", "quantity": 1, "payment": "434"}
</message>
No behaviour matched for message: <message to="systemagent@jabbim.pl/2" from="systemagent@jabbim.pl/4" thread="None" metadata={'buy': 'order'}>
{"item": "milk", "quantity": 1}
</message>
No behaviour matched for message: <message to="systemagent@jabbim.pl/3" from="systemagent@jabbim.pl/4" thread="None" metadata={'buy': 'order'}>
{"item": "milk", "quantity": 1}
</message>


systemagent@jabbim.pl/2 please milk
systemagent@jabbim.pl/3 please milk
no item now
Changing price from 483 to 434
proposing
Message received with content: {"item": "milk", "payment": "434"}
sending cash 434
systemagent@jabbim.pl/2 please milk
systemagent@jabbim.pl/3 please milk


No behaviour matched for message: <message to="systemagent@jabbim.pl/2" from="systemagent@jabbim.pl/6" thread="None" metadata={'buy': 'order'}>
{"item": "milk", "quantity": 1}
</message>
No behaviour matched for message: <message to="systemagent@jabbim.pl/3" from="systemagent@jabbim.pl/6" thread="None" metadata={'buy': 'order'}>
{"item": "milk", "quantity": 1}
</message>


systemagent@jabbim.pl/2 please milk
systemagent@jabbim.pl/3 please milk


In [21]:
factory.Stock.flatten()

,time,product,quantity,agent
0,2022-06-12 14:12:39.520837,milk,1,systemagent@jabbim.pl/2
1,2022-06-12 14:12:39.535929,milk,0,systemagent@jabbim.pl/2
2,2022-06-12 14:12:41.521003,milk,1,systemagent@jabbim.pl/2
3,2022-06-12 14:12:41.541026,milk,0,systemagent@jabbim.pl/2
4,2022-06-12 14:12:43.521150,milk,1,systemagent@jabbim.pl/2
...,...,...,...,...
67,2022-06-12 14:12:51.568144,milk,7,systemagent@jabbim.pl/6
68,2022-06-12 14:12:53.573649,milk,8,systemagent@jabbim.pl/6
69,2022-06-12 14:12:55.579087,milk,9,systemagent@jabbim.pl/6
70,2022-06-12 14:12:59.588882,milk,10,systemagent@jabbim.pl/6
